# Ollama + BQ Insert

In [ ]:
import subprocess
import json
import re

def estimate_nutrition_with_ollama(meal_description):
    prompt = f"""
    Estimate the total calories, protein (g), carbs (g), and fat (g) in this meal:

    "{meal_description}"

    Return ONLY a valid JSON object, no explanation, no markdown, just:
    {{
        "calories": 500,
        "protein_g": 25,
        "carbs_g": 40,
        "fat_g": 20
    }}
    """

    result = subprocess.run(
        ["ollama", "run", "llama2"],
        input=prompt.encode(),
        capture_output=True
    )

    raw_output = result.stdout.decode().strip()

    # Clean up markdown/code block formatting if present
    cleaned_output = re.sub(r"```(json)?", "", raw_output).strip()

    try:
        parsed = json.loads(cleaned_output)
        return parsed, raw_output
    except json.JSONDecodeError:
        print("❌ Could not parse LLM response — saving raw anyway.")
        return None, raw_output


#### Create BQ Table

In [52]:
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

def create_meals_table_if_needed():
    client = bigquery.Client()
    table_id = "vocal-spirit-372618.diet_data.meals"  # Replace with your own project/dataset

    schema = [
        bigquery.SchemaField("timestamp", "DATETIME", mode="REQUIRED"),
        bigquery.SchemaField("user_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("meal_description", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("calories", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("protein_g", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("carbs_g", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("fat_g", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("raw_response", "STRING", mode="REQUIRED"),
    ]

    try:
        client.get_table(table_id)
        print("✅ Table already exists.")
    except NotFound:
        table = bigquery.Table(table_id, schema=schema)
        client.create_table(table)
        print(f"✅ Created table {table_id}")


#### Load BQ Table

In [53]:
from datetime import datetime

def insert_meal_to_bq(user_id, meal, nutrition_data, raw_response):
    client = bigquery.Client()
    table_id = "vocal-spirit-372618.diet_data.meals"

    row = {
        "timestamp": datetime.utcnow().isoformat(),
        "user_id": user_id,
        "meal_description": meal,
        "calories": nutrition_data.get("calories"),
        "protein_g": nutrition_data.get("protein_g"),
        "carbs_g": nutrition_data.get("carbs_g"),
        "fat_g": nutrition_data.get("fat_g"),
        "raw_response": raw_response
    }

    errors = client.insert_rows_json(table_id, [row])
    if errors:
        print("❌ BigQuery insert errors:", errors)
    else:
        print("✅ Meal inserted into BigQuery.")


In [55]:
# Create table if needed (run once)
create_meals_table_if_needed()

# Log a meal
user_id = "blake@demo"
meal = "3 scrambled eggs with cheese, 2 slices of toast, and a cup of black coffee"

nutrition_data, raw_response = estimate_nutrition_with_ollama(meal)
insert_meal_to_bq(user_id, meal, nutrition_data or {}, raw_response)


✅ Table already exists.
✅ Meal inserted into BigQuery.
